In [1]:
!git clone https://github.com/bushuyeu/ece405-assignment1-basics.git

fatal: destination path 'ece405-assignment1-basics' already exists and is not an empty directory.


In [2]:
%cd ece405-assignment1-basics/

/content/ece405-assignment1-basics


In [3]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 2.65 KiB | 906.00 KiB/s, done.
From https://github.com/bushuyeu/ece405-assignment1-basics
   1fc363f..e06548d  main       -> origin/main
Updating 1fc363f..e06548d
Fast-forward
 .../ECE405_Homework1_ipynb_Pavel_Bushuyeu.ipynb    | 266 ++++++---------------
 1 file changed, 77 insertions(+), 189 deletions(-)


In [4]:
!git checkout main

Already on 'main'
Your branch is up to date with 'origin/main'.


No need for Conda environment on Colab

In [5]:
!pip install -e .'[test]'

Obtaining file:///content/ece405-assignment1-basics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for cs336-basics (pyproject.toml) ... done
  Created wheel for cs336-basics: filename=cs336_basics-1.0.6-py3-none-any.whl size=3234 sha256=b18d060760a0edb776e46debc7cf78a1e33ad6dcdfeecf431a45556c4636ac03
  Stored in directory: /tmp/pip-ephem-wheel-cache-1z7qx0ue/wheels/9c/8e/ad/47aa51c5b697fa6e79290e71856f3cf535f60484b164938fb6
Successfully built cs336-basics
  Attempting uninstall: cs336-basics
    Found existing installation: cs336-basics 1.0.6
    Uninstalling cs336-basics-1.0.6:
      Successfully uninstalled cs336-basics-1.0.6


In [6]:
!pytest tests/test_train_bpe.py

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0
rootdir: /content/ece405-assignment1-basics
configfile: pytest.ini
plugins: jaxtyping-0.3.7, langsmith-0.6.6, typeguard-4.4.4, anyio-4.12.1
collected 3 items                                                              

tests/test_train_bpe.py::test_train_bpe_speed PASSED                     [ 33%]
tests/test_train_bpe.py::test_train_bpe PASSED                           [ 66%]
tests/test_train_bpe.py::test_train_bpe_special_tokens PASSED            [100%]

============================== 3 passed in 1.99s ===============================


In [7]:
%cd /content

# Remove old data folder and start fresh
!rm -rf data
!mkdir -p data
%cd data

# Download TinyStories
!wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt
!wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-valid.txt

# Download OpenWebText sample
!wget https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_train.txt.gz
!gunzip owt_train.txt.gz
!wget https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_valid.txt.gz
!gunzip owt_valid.txt.gz

# Verify all downloads
import os
print("\n=== Download verification ===")
for f in ["TinyStoriesV2-GPT4-train.txt", "TinyStoriesV2-GPT4-valid.txt", "owt_train.txt", "owt_valid.txt"]:
    if os.path.exists(f):
        size = os.path.getsize(f)
        print(f"{f}: {size / 1e9:.2f} GB")
    else:
        print(f"{f}: MISSING!")

%cd /content/ece405-assignment1-basics


/content
/content/data
--2026-02-05 06:36:00--  https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt
Resolving huggingface.co (huggingface.co)... 3.165.160.11, 3.165.160.59, 3.165.160.12, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.11|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/645e8da96320b0efe40ade7a/02e40cc51c59a4bc6c51bd7bc9acda4316e208745be060558eaf500cd14e9f96?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27TinyStoriesV2-GPT4-train.txt%3B+filename%3D%22TinyStoriesV2-GPT4-train.txt%22%3B&response-content-type=text%2Fplain&Expires=1770276960&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzcwMjc2OTYwfX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjQ1ZThkYTk2MzIwYjBlZmU0MGFkZTdhLzAyZTQwY2M1MWM1OWE0YmM2YzUxYmQ3YmM5YWNkYTQzMTZlMjA4NzQ1YmUwNjA1NThlYWY1MDBjZDE0ZTlmOTZcXD9yZXNwb25zZ

In [ ]:
%cd /content/ece405-assignment1-basics

# Train BPE on TinyStories with timing and memory tracking
import time
import tracemalloc
from ece496b_basics import train_bpe

tracemalloc.start()
start_time = time.time()

vocab, merges = train_bpe(
    input_path="/content/data/TinyStoriesV2-GPT4-train.txt",
    vocab_size=10000,
    special_tokens=["<|endoftext|>"]
)

elapsed_time = time.time() - start_time
current_mem, peak_mem = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Training time: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")
print(f"Peak memory: {peak_mem / 1e9:.2f} GB")
print(f"Vocab size: {len(vocab)}")
print(f"Number of merges: {len(merges)}")

# Find the longest token
longest_token = max(vocab.values(), key=len)
print(f"\nLongest token: {longest_token!r}")
print(f"Longest token decoded: '{longest_token.decode('utf-8', errors='replace')}'")
print(f"Length: {len(longest_token)} bytes")

/content/ece405-assignment1-basics


: 

In [ ]:
# Save vocab and merges to disk
import json
import pickle
import os

os.makedirs("outputs", exist_ok=True)

with open("vocab.json", "w") as f:
    vocab_serializable = {k: v.hex() for k, v in vocab.items()}
    json.dump(vocab_serializable, f)

with open("merges.txt", "w") as f:
    for a, b in merges:
        f.write(f"{a.hex()} {b.hex()}\n")

with open("outputs/vocab_10k.pkl", "wb") as f:
    pickle.dump(vocab, f)
with open("outputs/merges_10k.pkl", "wb") as f:
    pickle.dump(merges, f)

print("Saved vocab.json, merges.txt, and pickle files")

In [ ]:
# Profile the training to answer: what part takes the most time?
import cProfile
import pstats
from io import StringIO

profiler = cProfile.Profile()
profiler.enable()

vocab2, merges2 = train_bpe(
    input_path="/content/data/TinyStoriesV2-GPT4-train.txt",
    vocab_size=10000,
    special_tokens=["<|endoftext|>"]
)

profiler.disable()

s = StringIO()
ps = pstats.Stats(profiler, stream=s).sort_stats('cumulative')
ps.print_stats(20)
print(s.getvalue())